## Cell-Free Genomics: Identify Enriched Ends

#### Ruby Froom, Campbell/Darst and Rock labs at Rockefeller University

The following pipeline downsamples, calls genomic coordinates where of enriched RNA ends (5' or 3', depending on the library preparation) occur. 

Other notebooks in `darst-campbell-lab` Github contain 5' or 3' end-specific interactive workflows for filtering of these coordinates, motif searching, and sequence-feature annotation. 
The goal of these notebooks is to reveal *cis* (promoter features for 5' ends; RNA structure and elemental pauses for 3' ends) and *trans* (TF binding regions) determinants of transcription regulation.

## Modules and functions

In [28]:
import subprocess, time
import pandas as pd
import numpy as np
import csv
from os import listdir
from os.path import exists
import scipy
from statsmodels.stats.multitest import fdrcorrection

In [29]:
# written by Peter Culviner, PhD to enable command-line access through Jupyter
def quickshell(command, print_output=True, output_path=None, return_output=False):
    process_output = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout = process_output.stdout.decode('utf-8')
    stderr = process_output.stderr.decode('utf-8')
    output_string = f'STDOUT:\n{stdout}\nSTDERR:\n{stderr}\n'
    if print_output:
        print('$ ' + command)
        print(output_string)
    if output_path is not None:
        with open(output_path, 'w') as f:
            f.write(output_string)
    if return_output:
        return stdout, stderr

Inputs for directory creation and pointing to files. 

The starting requirements for this pipeline are:
>**main_path**: the absolute path of the working directory. Update to reflect your own configuration.

>**input_csv_dir:** a directory containing `i7_sample_file` and `inline_sample_file` csv files (see below for file specifications) called `input_csv_files`.

In [31]:
# initializing locations of input .csv files, etc. 
main_path = '5enrich_CRP'
#main_path = '3enrich_NusAG'
main_path = 'gDNA'

input_csv_dir = f'{main_path}/input_csv_files'

#sample_table = pd.read_csv(f'{input_csv_dir}/inline_barcodes_5enrich.csv')
sample_table = pd.read_csv(f'{input_csv_dir}/inline_barcodes_3enrich.csv')
#sample_table = pd.read_csv(f'{input_csv_dir}/i7_barcodes_gDNA.csv')

,r1,r2,i7,title
0,frag_gDNA_3enrich1_S1_R1_001.fastq.gz,frag_gDNA_3enrich1_S1_R2_001.fastq.gz,1,gDNA1
1,frag_gDNA_3enrich2_S2_R1_001.fastq.gz,frag_gDNA_3enrich2_S2_R2_001.fastq.gz,2,gDNA2
2,frag_gDNA_3enrich3_S3_R1_001.fastq.gz,frag_gDNA_3enrich3_S3_R2_001.fastq.gz,3,gDNA3


The additional directories below are a suggested organization for subsequent processing steps in this notebook. Assumes that the mode of enriched end calling will be `bootstrap` (see __ section below).

The cells below will initialize these variables and create the directories inside the working directory.

In [34]:
# Directories for RNA processing

# Read prep dirs
alignments_dir = f'{main_path}/readPrep/dedup_alignments'
R2_alignments_dir = f'{main_path}/readPrep/R2_alignments'
spike_R2_alignments_dir = f'{main_path}/readPrep/spike_R2_alignments'

# Extracting single-bp info from alignments
identifyEnrichedEnds_dir = f'{main_path}/identifyEnrichedEnds'
downsample_dir = f'{identifyEnrichedEnds_dir}/downsampled_R2_bams'
bigWig_dir = f'{identifyEnrichedEnds_dir}/bigWigs'
ends_dir = f'{identifyEnrichedEnds_dir}/ends'
coverage_dir = f'{identifyEnrichedEnds_dir}/coverage'
bootstrap_calls_dir = f'{identifyEnrichedEnds_dir}/bootstrap_calls'
pre_blackList_dir = f'{identifyEnrichedEnds_dir}/consensus_calls_preBlackList'
post_blackList_dir = f'{identifyEnrichedEnds_dir}/consensus_calls_postBlackList'

# Same but for spike
spike_downsample_dir = f'{identifyEnrichedEnds_dir}/downsampled_spike_R2_bams'
spike_bigWig_dir = f'{identifyEnrichedEnds_dir}/bigWigs_spike'
spike_ends_dir = f'{identifyEnrichedEnds_dir}/ends_spike'
spike_coverage_dir = f'{identifyEnrichedEnds_dir}/cov_spike'
spike_bootstrap_calls_dir = f'{identifyEnrichedEnds_dir}/bootstrap_calls_spike'

In [ ]:
# Directories for gDNA processing
alignments_dir = f'{main_path}/readPrep/alignments'
R2_alignments_dir = f'{main_path}/readPrep/R2_alignments'
spike_R2_alignments_dir = f'{main_path}/readPrep/spike_R2_alignments'

# Extracting single-bp info from alignments
identifyEnrichedEnds_dir = f'{main_path}/identifyEnrichedEnds'
downsample_dir = f'{identifyEnrichedEnds_dir}/downsampled_R2_bams'
bigWig_dir = f'{identifyEnrichedEnds_dir}/bigWigs'
ends_dir = f'{identifyEnrichedEnds_dir}/ends'
coverage_dir = f'{identifyEnrichedEnds_dir}/coverage'
bootstrap_calls_dir = f'{identifyEnrichedEnds_dir}/bootstrap_calls'

In [ ]:
!mkdir $identifyEnrichedEnds_dir
!mkdir $downsample_dir
!mkdir $bigWig_dir
!mkdir $ends_dir
!mkdir $coverage_dir
!mkdir $bootstrap_calls_dir

!mkdir $spike_downsample_dir
!mkdir $spike_bigWig_dir
!mkdir $spike_ends_dir
!mkdir $spike_coverage_dir
!mkdir $spike_bootstrap_calls_dir

# Calling enriched ends

This section makes use of a script written by Mike Wolfe, PhD (Landick lab) to analyze single-bp resolution data.

Enriched ends are called based on a bootstrapping algorithm, where counts are shuffled within a user-defined window flanking every genomic coordinate for 10,000 iterations to manually assign p-values to every position in the genome (calculated as # iterations where random value > actual value, divided by 10,000).

In [4]:
# Indicate how many CPUs you want to use
processors = 10

## For both: err on the side of inclusive, because further filtering will be carried out
# Choose starting minimum coverage value to call an enriched transcript end
cov_cutoff = 2

# Choose maximum q-value to include a peak in the output of the bootstrap calling
alpha = 1

# Number of bp on either side of potential peak to consider for bootstrapping
window = 100

# Number of replicates per sample
replicates = 3

## Downsample R2 reads (experimental and spike)

In [7]:
read_depths = pd.read_csv(f'{R2_alignments_dir}/R2_read_counts.csv')

In [8]:
print_output = False
return_output = False

read_depths = pd.read_csv(f'{R2_alignments_dir}/R2_read_counts.csv')

# Then, set the sample_table to a dataframe that only contains the samples you'll be analyzing
# Change if need be (i.e. one sample has read depths that are too low)
# Here, the downsample depth is the minimum read depth across all samples. Can change if need be
downsample_depth = read_depths['R2_read_counts'].min()

In [ ]:
downsampled_depths = []

for row in sample_table.iterrows():
    _, sample_data = row
    
    original_depth = read_depths.loc[read_depths['Sample_Name'] == sample_data.title, 'R2_read_counts']
    downsample_command = f'samtools view -b -s {(downsample_depth/int(original_depth))} ' + \
                         f'{R2_alignments_dir}/{sample_data.title}_R2.bam > ' + \
                         f'{downsample_dir}/{sample_data.title}_downsample.bam'
    downsample = quickshell(downsample_command,
                            print_output = print_output,
                            return_output = return_output)
    
    # Index the downsampled R2 only bam
    index_downsample_command = f'samtools index {downsample_dir}/{sample_data.title}_downsample.bam'
    index_downsample = quickshell(index_downsample_command,
                                  print_output = print_output,
                                  return_output = return_output)
    
    count_downsample_command = f'samtools view -c ' + \
                                  f'{downsample_dir}/{sample_data.title}_downsample.bam'
    count_downsample = int(quickshell(count_downsample_command,
                                         print_output = False,
                                         return_output=True)[0].split('\n')[0])
    downsampled_depths.append([sample_data.title, count_downsample])
    
    # Spike downsampling
    spike_downsample_command = f'samtools view -b -s {(downsample_depth/int(original_depth))} ' + \
                         f'{spike_R2_alignments_dir}/{sample_data.title}_R2.bam > ' + \
                         f'{spike_downsample_dir}/{sample_data.title}_downsample.bam'
    spike_downsample = quickshell(spike_downsample_command,
                            print_output = print_output,
                            return_output = return_output)
    
    # Index the downsampled R2 only bam
    index_spike_downsample_command = f'samtools index {spike_downsample_dir}/{sample_data.title}_downsample.bam'
    index_spike_downsample = quickshell(index_spike_downsample_command,
                                  print_output = print_output,
                                  return_output = return_output)
    
    count_spike_downsample_command = f'samtools view -c ' + \
                                  f'{spike_downsample_dir}/{sample_data.title}_downsample.bam'
    count_spike_downsample = int(quickshell(count_spike_downsample_command,
                                         print_output = False,
                                         return_output=True)[0].split('\n')[0])
    downsampled_depths.append([sample_data.title, count_downsample, count_spike_downsample])
    
    print(f'Downsampling: {sample_data.title} done')
    
downsample_DF = pd.DataFrame(downsampled_depths, columns = ['Sample_Name','Downsampled_read_counts',
                                                           'Downsampled_spike_read_counts'])
downsample_DF.to_csv(f'{downsample_dir}/downsampled_depths.csv')

In [ ]:
# Check that spike read proportion was preserved
downsampled_depths = pd.read_csv(f'{downsample_dir}/downsampled_depths.csv')
previous_spike_reads = pd.read_csv(f'{spike_R2_alignments_dir}/R2_read_counts.csv')
previous_experimental_reads = pd.read_csv(f'{R2_alignments_dir}/R2_read_counts.csv')

downsampled_depths['Total starting R2 reads'] = previous_spike_reads['R2_read_counts'] + \
                                    previous_experimental_reads['R2_read_counts']
downsampled_depths['Total new R2 reads'] = downsampled_depths['Downsampled_read_counts'] + \
                                           downsampled_depths['Downsampled_spike_read_counts']

downsampled_depths['Previous spike proportion'] = previous_spike_reads['R2_read_counts'] / downsampled_depths['Total R2 reads']
downsampled_depths['New spike proportion'] = downsampled_depths['Downsampled_spike_read_counts'] / downsampled_depths['Total new R2 reads']
downsampled_depths.to_csv(f'{downsample_dir}/spike_proportion_check.csv')

## Generate single-bp resolution count data

In [3]:
# Generate single-bp coverage: both ends and total coverage

# Set print_output = True to see command-line output
# Set return_output = False if assigning commmand-line output to a variable
print_output = True
return_output = False

# Generate pull out ends of downsampled R2 reads
for row in sample_table.iterrows():
    _, sample_data = row
    # prepare input and output titles
    bam_input = f'{downsample_dir}/{sample_data.title}_downsample.bam'
    end_outputs = [
        f'{ends_dir}/{sample_data.title}_ends_plus.txt',
        f'{ends_dir}/{sample_data.title}_ends_minus.txt']
    # commands
    command_plus = f'bedtools genomecov -ibam {bam_input} -d -strand + -5 > {end_outputs[0]}'
    command_minus = f'bedtools genomecov -ibam {bam_input} -d -strand - -5 > {end_outputs[1]}'
    output_plus = quickshell(command_plus, print_output = print_output, return_output = return_output)
    output_minus = quickshell(command_minus, print_output = print_output, return_output = return_output)
    print(f'Single-bp end coverage: {sample_data.title} done')

# Generate single-bp coverage of downsampled R2 reads
for row in sample_table.iterrows():
    _, sample_data = row
    # prepare input and output titles
    bam_input = f'{downsample_dir}/{sample_data.title}_downsample.bam'
    end_outputs = [
        f'{coverage_dir}/{sample_data.title}_coverage_plus.txt',
        f'{coverage_dir}/{sample_data.title}_coverage_minus.txt']
    # commands
    command_plus = f'bedtools genomecov -ibam {bam_input} -d -strand + > {end_outputs[0]}'
    command_minus = f'bedtools genomecov -ibam {bam_input} -d -strand - > {end_outputs[1]}'
    output_plus = quickshell(command_plus, print_output = print_output, return_output = return_output)
    output_minus = quickshell(command_minus, print_output = print_output, return_output = return_output)
    print(f'Single-bp full coverage: {sample_data.title} done')

In [1]:
# Generate single-bp coverage of spike sample: both ends and total coverage

# Set print_output = True to see command-line output
# Set return_output = False if assigning commmand-line output to a variable
print_output = True
return_output = False

# Generate pull out ends of downsampled R2 reads
for row in sample_table.iterrows():
    _, sample_data = row
    # prepare input and output titles
    bam_input = f'{spike_downsample_dir}/{sample_data.title}_downsample.bam'
    end_outputs = [
        f'{spike_ends_dir}/{sample_data.title}_ends_plus.txt',
        f'{spike_ends_dir}/{sample_data.title}_ends_minus.txt']
    # commands
    command_plus = f'bedtools genomecov -ibam {bam_input} -d -strand + -5 > {end_outputs[0]}'
    command_minus = f'bedtools genomecov -ibam {bam_input} -d -strand - -5 > {end_outputs[1]}'
    output_plus = quickshell(command_plus, print_output = print_output, return_output = return_output)
    output_minus = quickshell(command_minus, print_output = print_output, return_output = return_output)
    print(f'Single-bp spike end coverage: {sample_data.title} done')

# Generate single-bp coverage of downsampled R2 reads
for row in sample_table.iterrows():
    _, sample_data = row
    # prepare input and output titles
    bam_input = f'{spike_downsample_dir}/{sample_data.title}_downsample.bam'
    end_outputs = [
        f'{spike_coverage_dir}/{sample_data.title}_coverage_plus.txt',
        f'{spike_coverage_dir}/{sample_data.title}_coverage_minus.txt']
    # commands
    command_plus = f'bedtools genomecov -ibam {bam_input} -d -strand + > {end_outputs[0]}'
    command_minus = f'bedtools genomecov -ibam {bam_input} -d -strand - > {end_outputs[1]}'
    output_plus = quickshell(command_plus, print_output = print_output, return_output = return_output)
    output_minus = quickshell(command_minus, print_output = print_output, return_output = return_output)
    print(f'Single-bp spike full coverage: {sample_data.title} done')

## Generate bigWig inputs for enriched end calling

In [2]:
# Set print_output = True to see command-line output
# Set return_output = False if assigning commmand-line output to a variable
print_output = False
return_output = False

Mtb_region = 'Eco_Mtb:4641653:9053361'

# Generate bigWig files for peak calling
for row in inline_table.iterrows():
    _, sample_data = row
    
    bigwig_plus_command = f'bamCoverage --bam {downsample_dir}/{sample_data.title}_downsample.bam ' + \
                          f'--numberOfProcessors {processors} --region {ends_region} ' + \
                          f'--skipNonCoveredRegions --skipNAs ' + \
                          f'--outFileName {bigWig_dir}/{sample_data.title}_plus.bw --outFileFormat bigwig ' + \
                          f'--binSize 1 --filterRNAstrand forward --Offset 1' 
    bigwig_minus_command = f'bamCoverage --bam {downsample_dir}/{sample_data.title}_downsample.bam ' + \
                           f'--numberOfProcessors {processors} --region {ends_region} '+ \
                           f'--skipNonCoveredRegions --skipNAs ' + \
                           f'--outFileName {bigWig_dir}/{sample_data.title}_minus.bw --outFileFormat bigwig ' + \
                           f'--binSize 1 --filterRNAstrand reverse --Offset 1' 
 
    bigwig_plus = quickshell(bigwig_plus_command, print_output = print_output, return_output = return_output)
    print(f'bigWig generation (calling input): {sample_data.title} plus done')
    bigwig_minus = quickshell(bigwig_minus_command, print_output = print_output, return_output = return_output)
    print(f'bigWig generation (calling input): {sample_data.title} minus done')

In [3]:
# Spike
# Set print_output = True to see command-line output
# Set return_output = False if assigning commmand-line output to a variable
print_output = False
return_output = False

Eco_region = 'Eco_Mtb:1-4641652'

# Generate bigWig files for peak calling
for row in inline_table.iterrows():
    _, sample_data = row
    
    bigwig_plus_command = f'bamCoverage --bam {spike_downsample_dir}/{sample_data.title}_downsample.bam ' + \
                          f'--numberOfProcessors {processors} --region {ends_region} ' + \
                          f'--skipNonCoveredRegions --skipNAs ' + \
                          f'--outFileName {spike_bigWig_dir}/{sample_data.title}_plus.bw --outFileFormat bigwig ' + \
                          f'--binSize 1 --filterRNAstrand forward --Offset 1' 
    bigwig_minus_command = f'bamCoverage --bam {spike_downsample_dir}/{sample_data.title}_downsample.bam ' + \
                           f'--numberOfProcessors {processors} --region {ends_region} '+ \
                           f'--skipNonCoveredRegions --skipNAs ' + \
                           f'--outFileName {spike_bigWig_dir}/{sample_data.title}_minus.bw --outFileFormat bigwig ' + \
                           f'--binSize 1 --filterRNAstrand reverse --Offset 1' 
 
    bigwig_plus = quickshell(bigwig_plus_command, print_output = print_output, return_output = return_output)
    print(f'spike bigWig generation (calling input): {sample_data.title} plus done')
    bigwig_minus = quickshell(bigwig_minus_command, print_output = print_output, return_output = return_output)
    print(f'spike bigWig generation (calling input): {sample_data.title} minus done')

## Call enriched ends

In [ ]:
# Experimental samples

for row in sample_table.iterrows():
    _, sample_data = row
    end_enrich_command = f'python3 NETseq_pause_calling.py Genomewide ' + \
                         f'{bigWig_dir}/{sample_data.title}_plus.bw ' + \
                         f'{bigWig_dir}/{sample_data.title}_minus.bw {window} --p {processors} ' + \
                         f'--method bootstrap --filter cov --cov_cutoff {cov_cutoff} ' + \
                         f'--alpha {alpha} > {bootstrap_calls_dir}/{sample_data.title}_calls_alpha{alpha}.txt'
    end_enrich = quickshell(end_enrich_command, print_output = False, return_output = False)
    print(f'Bootstrap calling: {sample_data.title} done')

In [ ]:
# Spike samples

for row in sample_table.iterrows():
    _, sample_data = row
    end_enrich_command = f'python3 NETseq_pause_calling.py Genomewide ' + \
                         f'{spike_bigWig_dir}/{sample_data.title}_plus.bw ' + \
                         f'{spike_bigWig_dir}/{sample_data.title}_minus.bw {window} --p {processors} ' + \
                         f'--method bootstrap --filter cov --cov_cutoff {cov_cutoff} ' + \
                         f'--alpha {alpha} > {spike_bootstrap_calls_dir}/{sample_data.title}_calls_alpha{alpha}.txt'
    end_enrich = quickshell(end_enrich_command, print_output = False, return_output = False)
    print(f'Spike bootstrap calling: {sample_data.title} done')

# Extract spike consensus calls

Use CellFreeGenomics_thresholdSelection to systematically test different count thresholds for experimental data.

Usually, spike data only comprise 5-10% of the sample and are just used for size factor estimation,
so the lower count threshold of 2 works well to get enough peaks for size factor estimation in DESeq2.

As a result, spike consensus calls can be easily generated here.

In [41]:
def consensus_peaks(condition, replicates, count_cutoff):
    """
    Generates a consensus dataframe of called peaks from a list of replicates.
    condition is a string containing the name of the experimental condition, e.g. NusG.
    replicates: number of replicates.
    count_cutoff: >= cutoff value in 'ends' column.
    """
    # Initialize empty lists
    plus_replicate_DF_list = []
    minus_replicate_DF_list = []
    plus_ends_list = []
    minus_ends_list = []
    
    replicate_numbers_DF = sample_table.loc[sample_table['condition'] == condition,]
    replicate_numbers_DF.reset_index(inplace = True)
    replicate_numbers = replicate_numbers_DF['replicate']
    
    # For each replicate:
    for i in range(replicates):
        
        # Read in the called peaks for each replicate
        next_DF = pd.read_table(f'{spike_bootstrap_calls_dir}/{condition}{replicate_numbers[i]}' + \
                                f'_calls_alpha{alpha}.txt')
        # Add the replicate as a column
        next_DF['replicate'] = replicate_numbers[i]
        
        # Filter for ends that have a certain number of counts
        next_DF_highCov = next_DF.loc[next_DF['count'] >= 2,]
        
        # Adjust p-values again after applying coverage filter
        _, qvals = fdrcorrection(next_DF_highCov['pvalue'], method = "i")
        next_DF_highCov['qvalue_updated'] = qvals
        
        # Filter for significant peaks (q-val = post-multiple hypothesis testing correction)
        next_DF_sig = next_DF_highCov.loc[next_DF_highCov['qvalue_updated'] <= sig_cutoff,]
        
        # Separate + and - coordinates
        next_DF_sig_plus = next_DF_sig.loc[next_DF_sig['strand'] == '+',]
        next_DF_sig_minus = next_DF_sig.loc[next_DF_sig['strand'] == '-',]
        
        # Append plus and minus coordinates to empty lists
        plus_replicate_DF_list.append(next_DF_sig_plus)
        minus_replicate_DF_list.append(next_DF_sig_minus)
        plus_ends_list.append(set(next_DF_sig_plus['end']))
        minus_ends_list.append(set(next_DF_sig_minus['end']))
        
    # Put all individual replicates per condition into one large DataFrame
    plus_replicate_DFs = pd.concat(plus_replicate_DF_list)
    minus_replicate_DFs = pd.concat(minus_replicate_DF_list)
    
    # Only include enriched ends that were called in all replicates
    plus_ends_intersection = set.intersection(*plus_ends_list)
    minus_ends_intersection = set.intersection(*minus_ends_list)
    plus_replicate_DF_allReps = plus_replicate_DFs.loc[plus_replicate_DFs['end'].isin(plus_ends_intersection),]
    minus_replicate_DF_allReps = minus_replicate_DFs.loc[minus_replicate_DFs['end'].isin(minus_ends_intersection),]
    
    # Combine + and - DFs
    replicate_DF_allReps = pd.concat([plus_replicate_DF_allReps, minus_replicate_DF_allReps])
    
    replicate_DF_allReps['condition'] = condition
    
    # Write out final dataframe
    replicate_DF_allReps.to_csv(f'{spike_bootstrap_calls_dir}/' + \
                    f'spike_{condition}_consensus_calls_{replicates}reps.csv')
    print(f'Generating consensus spike peaks: {condition} done')
#    return replicate_DF_allReps

In [9]:
sample_table['condition'] = sample_table['title'].str[:-1]
sample_table['replicate'] = sample_table['title'].str[-1:]
condition_list = sample_table['condition'].unique()

for condition in condition_list:
    consensus_peaks(condition, replicates, cov_cutoff)